In [1]:
import bt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bt.core import Algo


In [2]:
data = pd.read_excel('./sp500_adj_close.xlsx',header=0, skiprows=1)
data = data[1:]
data.columns = ['datetime'] + data.columns[1:].to_list()
data['datetime'] = pd.to_datetime(data['datetime'])
data = data.set_index('datetime')
# special
data.drop(pd.to_datetime('2017-09-04'),inplace=True)
# data = data.loc['2018-01-01':]
data.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-04,20.545469,4.496876,39.103363,6.593423,NaN,22.512976,8.74,20.074522,33.718582,37.090000,...,19.830650,46.349365,32.941868,14.571863,NaN,20.328438,55.501415,28.670000,11.726357,NaN
2010-01-05,20.322289,5.005957,38.870968,6.604822,NaN,22.352354,8.53,19.912334,33.926979,37.700001,...,19.580618,46.530319,32.550251,14.588748,NaN,20.258917,57.258369,28.620001,12.139812,NaN
2010-01-06,20.250088,4.798555,39.209885,6.499765,NaN,22.141001,8.40,20.022911,34.287651,37.619999,...,19.447794,46.932491,32.764709,14.453672,NaN,20.114088,57.239891,28.400000,13.195447,NaN
2010-01-07,20.223829,4.939965,39.200214,6.487749,NaN,21.785934,8.40,20.188793,34.255581,36.889999,...,19.252451,46.785034,33.193615,14.521203,NaN,20.108294,58.552982,27.690001,14.673337,NaN
2010-01-08,20.217272,4.845690,39.355148,6.530882,NaN,22.022644,8.23,20.292002,34.119335,36.689999,...,19.533739,46.597363,33.193615,14.470551,NaN,20.114088,57.323116,27.600000,14.435819,NaN


In [3]:
benchmark_data = pd.read_excel('./sp_500_^SPGC_adj_close.xlsx')
benchmark_data['Date'] = pd.to_datetime(benchmark_data['Date'])
benchmark_data = benchmark_data.set_index('Date')
benchmark_data.head()

,Adj Close
Date,
2010-01-04,1132.989990
2010-01-05,1136.520020
2010-01-06,1137.140015
2010-01-07,1141.689941
2010-01-08,1144.979980


In [4]:
from finquant.portfolio import build_portfolio
from finquant.moving_average import sma, ema


class SelectAll_DIY(Algo):

    def __init__(self, include_no_data=False, include_negative=False):
        super(SelectAll_DIY, self).__init__()
        self.include_no_data = include_no_data
        self.include_negative = include_negative

    def __call__(self, target):
        if self.include_no_data:
            target.temp['selected'] = target.universe.columns
        else:
            universe = target.universe.loc[target.now].dropna()

            results = target.universe.apply(self.screen, benchmark_ann_ret=0.1).T
            select_list = results[results['meet_criterion']== True].index.to_list()

            target.temp['selected'] = select_list
            # print("-----target.temp['selected']",target.temp['selected'])
            # if self.include_negative:
            #     target.temp['selected'] = list(universe.index)
            # else:
            #     target.temp['selected'] = list(universe[universe > 0].index)
            return True



    # STRATEGY M&M MODEL.  #RANKS MY ESG SCORES.

    # 1. The current stock price is above both the 150-day and 200-day moving average
    # 2. The 150-day moving average is above the 200-day moving average
    # 3. The 200-day moving average line is trending up for at least 1 month
    # 4. The 50-day moving average is above both the 150-day and 200-day moving averages
    # 5. The current stock price is trading above the 50-day moving average
    # 6. The current stock price is at least 30% above it’s 52-week low
    # 7. The current stock price is within at least 25% of its 52-week high
    # 8. The relative strength ranking (as reported in Investor’s Business Daily) is no less than 70. Preferably in the 80s or 90s.

    # select stocks function

    def screen(self, close: pd.Series, benchmark_ann_ret: float) -> pd.Series:
        # EMA 50,150,200
        ema_50 = ema(close, 50).iloc[-1]
        ema_150 = ema(close, 150).iloc[-1]
        ema_200 = ema(close, 200).iloc[-1]

        # The 20-day movement of the 200-day moving average is smooth to determine whether the 200-day moving average is rising
        ema_200_smooth = ema(ema(close, 200), 20).iloc[-1]

        # Closing 52-week highs and 52-week lows
        high_52week = close.rolling(52 * 5).max().iloc[-1]
        low_52week = close.rolling(52 * 5).min().iloc[-1]

        # Recent closing
        cl = float(close.iloc[-1])

        # 1. The current stock price is above both the 150-day and 200-day moving average
        if cl > ema_150 and cl > ema_200:
            condition_1 = True
        else:
            condition_1 = False

        # 2. The 150-day moving average is above the 200-day moving average
        if ema_150 > ema_200:
            condition_2 = True
        else:
            condition_2 = False

        # 3. The 200-day moving average line is trending up for at least 1 month
        if ema_200 > ema_200_smooth:
            condition_3 = True
        else:
            condition_3 = False

        # 4. The 50-day moving average is above both the 150-day and 200-day moving averages
        if ema_50 > ema_150 and ema_50 > ema_200:
            condition_4 = True
        else:
            condition_4 = False

        # 5. The current stock price is trading above the 50-day moving average
        if cl > ema_50:
            condition_5 = True
        else:
            condition_5 = False

        # 6. The current stock price is at least 30% above it’s 52-week low
        if cl >= low_52week * 1.3:
            condition_6 = True
        else:
            condition_6 = False

        # 7. The current stock price is within at least 25% of its 52-week high
        if cl >= high_52week * 0.75 and cl <= high_52week * 1.25:
            condition_7 = True
        else:
            condition_7 = False

        # 8. The relative strength ranking (as reported in Investor’s Business Daily) is no less than 70. Preferably in the 80s or 90s.
        rs = close.pct_change(
            252).iloc[-1] / float(benchmark_ann_ret) * 100
        if rs >= 70:
            condition_8 = True
        else:
            condition_8 = False

        # 判断股票是否符合标准
        if (condition_1 and condition_2 and condition_3 and
            condition_4 and condition_5 and condition_6 and
                condition_7 and condition_8):
            meet_criterion = True
        else:
            meet_criterion = False

        out = {
            "rs": round(rs, 2),
            "close": cl,
            "ema_50": ema_50,
            "ema_150": ema_150,
            "ema_200": ema_200,
            "high_52week": high_52week,
            "low_52week": low_52week,
            "meet_criterion": meet_criterion
        }

        return pd.Series(out)


In [5]:

class WeighEqually_DIY(Algo):

    def __init__(self,optimial_type=0):
        super(WeighEqually_DIY, self).__init__()
        self.optimial_type = optimial_type

    def __call__(self, target):
        selected = target.temp['selected']
        n = len(selected)

        if n == 0:
            target.temp['weights'] = {}
        else:
            opt_weights = self.weight_portfolio(target.universe, selected, self.optimial_type)
            
            lst = opt_weights.index.tolist()
            opt_weights = opt_weights['Allocation'].to_list()

            target.temp['selected'] = lst
            target.temp['weights'] = {lst[i]: opt_weights[i] for i in range(len(lst))}

        return True


    def weight_portfolio(self, data, selected_list,optimial_type):


        if len(selected_list) == 0:
            return selected_list

        pf = build_portfolio(data=data[selected_list])

        # 1. set the new value(s)
        pf.freq = 252
        pf.risk_free_rate = 0.1

        
        opt_weights = None
        if optimial_type == 0:
            opt_weights = pf.ef_minimum_volatility()
        else:
            opt_weights = pf.ef_maximum_sharpe_ratio()
            
        
        return opt_weights

In [6]:
# create the strategy
s_mv = bt.Strategy('minimum volatility', [bt.algos.RunYearly(),
                       SelectAll_DIY(),
                       WeighEqually_DIY(optimial_type=0),
                       bt.algos.Rebalance()])
# create a backtest and run it
test_mv = bt.Backtest(s_mv, data)

In [7]:
# create the strategy
s_msr = bt.Strategy('maximum sharpe ratio', [bt.algos.RunYearly(),
                       SelectAll_DIY(),
                       WeighEqually_DIY(optimial_type=1),
                       bt.algos.Rebalance()])
# create a backtest and run it
test_msr = bt.Backtest(s_msr, data)
res = bt.run(test_mv, test_msr)

KeyboardInterrupt: 

In [ ]:
# res2 plots here include both s1 and s2 info
res.plot()

In [8]:
res.display()

Stat                 maximum sharpe ratio
-------------------  ----------------------
Start                2018-01-01
End                  2021-02-04
Risk-free rate       0.00%

Total Return         120.92%
Daily Sharpe         1.08
Daily Sortino        1.62
CAGR                 29.20%
Max Drawdown         -38.88%
Calmar Ratio         0.75

MTD                  5.03%
3m                   29.79%
6m                   32.31%
YTD                  7.98%
1Y                   51.50%
3Y (ann.)            30.26%
5Y (ann.)            -
10Y (ann.)           -
Since Incep. (ann.)  29.20%

Daily Sharpe         1.08
Daily Sortino        1.62
Daily Mean (ann.)    29.34%
Daily Vol (ann.)     27.08%
Daily Skew           -0.48
Daily Kurt           13.94
Best Day             13.86%
Worst Day            -11.97%

Monthly Sharpe       1.45
Monthly Sortino      3.21
Monthly Mean (ann.)  27.72%
Monthly Vol (ann.)   19.06%
Monthly Skew         -0.06
Monthly Kurt         0.42
Best Month           12.75%
Worst M

In [21]:
res.plot().get_figure().savefig("output.png")